In [8]:
# import numpy as np
# import pandas as pd
# import sqlite3

# df = pd.read_csv("data\sales_data_sample.csv", encoding = "ISO-8859-1")

# conn = sqlite3.connect('data\sales_db.sqlite')
# cursor = conn.cursor()

# # Define the CREATE TABLE SQL statement
# create_table_sql = """
# CREATE TABLE IF NOT EXISTS Orders (
#     ORDERNUMBER INT,
#     QUANTITYORDERED INT,
#     PRICEEACH DECIMAL(10, 2),
#     ORDERLINENUMBER INT,
#     SALES DECIMAL(10, 2),
#     ORDERDATE DATE,
#     STATUS TEXT,
#     QTR_ID INT,
#     MONTH_ID INT,
#     YEAR_ID INT,
#     PRODUCTLINE TEXT,
#     MSRP INT,
#     PRODUCTCODE TEXT,
#     CUSTOMERNAME TEXT,
#     PHONE TEXT,
#     ADDRESSLINE1 TEXT,
#     ADDRESSLINE2 TEXT,
#     CITY TEXT,
#     STATE TEXT,
#     POSTALCODE TEXT,
#     COUNTRY TEXT,
#     TERRITORY TEXT,
#     CONTACTLASTNAME TEXT,
#     CONTACTFIRSTNAME TEXT,
#     DEALSIZE TEXT
# );
# """

# # cursor.execute(create_table_sql)
# # conn.commit()
# print("Table 'Orders' created successfully.")

# df.to_sql('sales_orders', conn, if_exists='replace', index = False)

# cursor.execute('''
# SELECT CUSTOMERNAME FROM sales_orders LIMIT 100
#           ''')

# for row in cursor.fetchall():
#     print (row)

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain
from keys import openai_api_key, serp_api_key, neondb_uri
from langchain.prompts.prompt import PromptTemplate
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

import warnings
os.environ["OPENAI_API_KEY"] = openai_api_key
warnings.filterwarnings("ignore")

In [6]:
# llm = OpenAI(temperature=0)
# llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
input_db = SQLDatabase.from_uri('sqlite:///data/sales_db.sqlite')
toolkit = SQLDatabaseToolkit(db=input_db, llm=OpenAI(temperature=0))

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

# db_chain = SQLDatabaseChain(llm = llm,
#                             database = input_db,
#                             verbose=True,
#                             use_query_checker=True)

In [7]:


prompt = """You are a SQL expert. Given an input question, first create a syntactically correct redshift query to run, then look at the results of the query and return the answer.
Never query for all columns from a table. You must query only the columns that are needed to answer the question.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

Question: {question}"""

# prompt = """You are a SQL expert. Given an input question, first create a syntactically correct redshift query to run, then look at the results of the query and return the answer.
# Never query for all columns from a table. You must query only the columns that are needed to answer the question.
# Use the following format:

# Only use the following tables:

# {table_info}

# Question: {question}"""

# PROMPT = PromptTemplate(
#     input_variables=["question", "table_info"], template= DEFAULT_TEMPLATE
# )

In [8]:
# agent_executor.run(prompt.format(question="Describe the sales_orders table", table_info="sales_orders"))
agent_executor.run(prompt.format(question="Show first row from the table", table_info="sales_orders"))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: sales_orders
Thought: I should query the schema of the sales_orders table
Action: sql_db_schema
Action Input: "sales_orders"
Observation: 
CREATE TABLE sales_orders (
	"ORDERNUMBER" INTEGER, 
	"QUANTITYORDERED" INTEGER, 
	"PRICEEACH" REAL, 
	"ORDERLINENUMBER" INTEGER, 
	"SALES" REAL, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" INTEGER, 
	"MONTH_ID" INTEGER, 
	"YEAR_ID" INTEGER, 
	"PRODUCTLINE" TEXT, 
	"MSRP" INTEGER, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)

/*
3 rows from sales_orders table:
ORDERNUMBER	QUANTITYORDERED	PRICEEACH	ORDERLINENUMBER	SALES	ORDERDATE	STATUS	QTR_ID	MONTH_ID	YEAR_ID	PRODUCTLINE	MSRP	PRODUCTCODE	CUSTOMERNAME	PHONE	ADDRESSLINE1	ADDRESSLIN

"The first row from the sales_orders table is (10107, 30, 95.7, 2, 2871.0, '2/24/2003 0:00', 'Shipped', 1, 2, 2003, 'Motorcycles', 95, 'S10_1678', 'Land of Toys Inc.', '2125557818', '897 Long Airport Avenue', None, 'NYC', 'NY', '10022', 'USA', None, 'Yu', 'Kwai', 'Small')."

In [5]:
agent_executor.run(prompt.format(question="How many records are present in sales_orders?", table_info="sales_orders"))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Orders, sales_orders
Thought: I should query the schema of the sales_orders table to see what columns I can use.
Action: sql_db_schema
Action Input: "sales_orders"
Observation: 
CREATE TABLE sales_orders (
	"ORDERNUMBER" INTEGER, 
	"QUANTITYORDERED" INTEGER, 
	"PRICEEACH" REAL, 
	"ORDERLINENUMBER" INTEGER, 
	"SALES" REAL, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" INTEGER, 
	"MONTH_ID" INTEGER, 
	"YEAR_ID" INTEGER, 
	"PRODUCTLINE" TEXT, 
	"MSRP" INTEGER, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)

/*
3 rows from sales_orders table:
ORDERNUMBER	QUANTITYORDERED	PRICEEACH	ORDERLINENUMBER	SALES	ORDERDATE	STATUS	QTR_ID	MONTH_ID	YEAR_ID	PRODUCTLINE	MSRP	PRODUCTCODE	CUS

'There are 2823 records in the sales_orders table.'

In [12]:
agent_executor.run(prompt.format(question="Create a sql query to show Which customer has placed the most orders?", table_info="sales_orders"))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: sales_orders
Thought: I should query the schema of the sales_orders table to see what columns I can use.
Action: sql_db_schema
Action Input: "sales_orders"
Observation: 
CREATE TABLE sales_orders (
	"ORDERNUMBER" INTEGER, 
	"QUANTITYORDERED" INTEGER, 
	"PRICEEACH" REAL, 
	"ORDERLINENUMBER" INTEGER, 
	"SALES" REAL, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" INTEGER, 
	"MONTH_ID" INTEGER, 
	"YEAR_ID" INTEGER, 
	"PRODUCTLINE" TEXT, 
	"MSRP" INTEGER, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)

/*
3 rows from sales_orders table:
ORDERNUMBER	QUANTITYORDERED	PRICEEACH	ORDERLINENUMBER	SALES	ORDERDATE	STATUS	QTR_ID	MONTH_ID	YEAR_ID	PRODUCTLINE	MSRP	PRODUCTCODE	CUSTOMERNAM

'Tokyo Collectables, Ltd has placed the most orders with 32 orders.'

In [9]:
agent_executor.run(prompt.format(question="Which customer has made the largest deals in the APAC territory?", table_info=""))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: sales_orders
Thought: I should query the schema of the sales_orders table to see what columns I can use.
Action: sql_db_schema
Action Input: "sales_orders"
Observation: 
CREATE TABLE sales_orders (
	"ORDERNUMBER" INTEGER, 
	"QUANTITYORDERED" INTEGER, 
	"PRICEEACH" REAL, 
	"ORDERLINENUMBER" INTEGER, 
	"SALES" REAL, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" INTEGER, 
	"MONTH_ID" INTEGER, 
	"YEAR_ID" INTEGER, 
	"PRODUCTLINE" TEXT, 
	"MSRP" INTEGER, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)

/*
3 rows from sales_orders table:
ORDERNUMBER	QUANTITYORDERED	PRICEEACH	ORDERLINENUMBER	SALES	ORDERDATE	STATUS	QTR_ID	MONTH_ID	YEAR_ID	PRODUCTLINE	MSRP	PRODUCTCODE	CUSTOMERNAM

'The customer with the largest deals in the APAC territory is Australian Collectors, Co. with a deal of 9774.03.'

In [11]:
agent_executor.run(prompt.format(question="Which customer has made the largest deals in the APAC territory using the DEALSIZE column?", table_info=""))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: sales_orders
Thought: I should query the schema of the sales_orders table to see what columns I can use.
Action: sql_db_schema
Action Input: "sales_orders"
Observation: 
CREATE TABLE sales_orders (
	"ORDERNUMBER" INTEGER, 
	"QUANTITYORDERED" INTEGER, 
	"PRICEEACH" REAL, 
	"ORDERLINENUMBER" INTEGER, 
	"SALES" REAL, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" INTEGER, 
	"MONTH_ID" INTEGER, 
	"YEAR_ID" INTEGER, 
	"PRODUCTLINE" TEXT, 
	"MSRP" INTEGER, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)

/*
3 rows from sales_orders table:
ORDERNUMBER	QUANTITYORDERED	PRICEEACH	ORDERLINENUMBER	SALES	ORDERDATE	STATUS	QTR_ID	MONTH_ID	YEAR_ID	PRODUCTLINE	MSRP	PRODUCTCODE	CUSTOMERNAM

"Anna's Decorations, Ltd has made the largest deals in the APAC territory."

In [10]:
agent_executor.run(prompt.format(question="On which month of any year were most orders were placed?", table_info="sales_orders"))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: sales_orders
Thought: I should query the schema of the sales_orders table to see what columns I can use.
Action: sql_db_schema
Action Input: "sales_orders"
Observation: 
CREATE TABLE sales_orders (
	"ORDERNUMBER" INTEGER, 
	"QUANTITYORDERED" INTEGER, 
	"PRICEEACH" REAL, 
	"ORDERLINENUMBER" INTEGER, 
	"SALES" REAL, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" INTEGER, 
	"MONTH_ID" INTEGER, 
	"YEAR_ID" INTEGER, 
	"PRODUCTLINE" TEXT, 
	"MSRP" INTEGER, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)

/*
3 rows from sales_orders table:
ORDERNUMBER	QUANTITYORDERED	PRICEEACH	ORDERLINENUMBER	SALES	ORDERDATE	STATUS	QTR_ID	MONTH_ID	YEAR_ID	PRODUCTLINE	MSRP	PRODUCTCODE	CUSTOMERNAM

'The month with the most orders is November.'

In [13]:
agent_executor.run("Create a sql query to show Which customer has placed the most orders?")



> Entering new AgentExecutor chain...
Thought: I should use the sql_db_list_tables tool to get a list of tables in the database.
Action: sql_db_list_tables
Action Input: ""
Observation: sales_orders
Thought: I should use the sql_db_schema tool to get the schema of the sales_orders table.
Action: sql_db_schema
Action Input: "sales_orders"
Observation: 
CREATE TABLE sales_orders (
	"ORDERNUMBER" INTEGER, 
	"QUANTITYORDERED" INTEGER, 
	"PRICEEACH" REAL, 
	"ORDERLINENUMBER" INTEGER, 
	"SALES" REAL, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" INTEGER, 
	"MONTH_ID" INTEGER, 
	"YEAR_ID" INTEGER, 
	"PRODUCTLINE" TEXT, 
	"MSRP" INTEGER, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)

/*
3 rows from sales_orders table:
ORDERNUMBER	QUANTITYORDERED	PRICEEACH	ORDERLINENUMBE

'Euro Shopping Channel has placed the most orders with 259 orders.'